In [207]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [208]:
seasons = pd.read_csv("all_seasons.csv")
top50 = pd.read_csv("Top50.csv")

In [209]:
common = top50.merge(right=seasons, left_on="PLAYER", right_on="player_name", how="left").drop(columns=["index_x"])
common["team-year"] = common["team_abbreviation"] + "-" + common["season"]
common

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team-year
0,Joel Embiid,PHI,28,68,45,23,33.8,30.6,9.8,19.6,...,7.8,2.1,3.2,0.092,0.247,0.363,0.584,0.174,2016-17,PHI-2016-17
1,Joel Embiid,PHI,28,68,45,23,33.8,30.6,9.8,19.6,...,11.0,3.2,11.7,0.078,0.255,0.330,0.573,0.182,2017-18,PHI-2017-18
2,Joel Embiid,PHI,28,68,45,23,33.8,30.6,9.8,19.6,...,13.6,3.7,7.6,0.075,0.292,0.327,0.593,0.185,2018-19,PHI-2018-19
3,Joel Embiid,PHI,28,68,45,23,33.8,30.6,9.8,19.6,...,11.6,3.0,5.2,0.091,0.279,0.315,0.590,0.178,2019-20,PHI-2019-20
4,Joel Embiid,PHI,28,68,45,23,33.8,30.6,9.8,19.6,...,10.6,2.8,12.0,0.078,0.254,0.352,0.636,0.156,2020-21,PHI-2020-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Gary Trent Jr.,TOR,23,70,44,26,35.0,18.3,6.4,15.5,...,0.7,0.3,-4.7,0.010,0.080,0.213,0.377,0.057,2018-19,POR-2018-19
408,Gary Trent Jr.,TOR,23,70,44,26,35.0,18.3,6.4,15.5,...,1.6,1.0,-1.4,0.017,0.052,0.150,0.587,0.064,2019-20,POR-2019-20
409,Gary Trent Jr.,TOR,23,70,44,26,35.0,18.3,6.4,15.5,...,2.6,1.4,-1.8,0.014,0.069,0.204,0.534,0.067,2020-21,TOR-2020-21
410,Gary Trent Jr.,TOR,23,70,44,26,35.0,18.3,6.4,15.5,...,2.7,2.0,3.3,0.010,0.070,0.212,0.546,0.085,2021-22,TOR-2021-22


In [210]:
stats14 = common[((common["season"].str.slice(start=0,stop=4)).astype(int) > 2013) & ((common["season"].str.slice(start=0,stop=4)).astype(int) < 2021)]
curr_age = stats14.groupby("PLAYER")["age"].max()
stats14 = stats14.groupby("PLAYER").mean()
#.to_csv("stats16.csv")
stats14["age"] = curr_age + 1
stats14 = stats14.drop(columns=["AGE"])
stats14.to_csv("stats14.csv")
stats14["net_rating"].sort_values()

/var/folders/l7/bpy022dn1q71ws_q0dtvqst00000gn/T/ipykernel_34550/3011516107.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  stats14 = stats14.groupby("PLAYER").mean()


PLAYER
Julius Randle             -14.885714
Darius Garland             -7.650000
Zach LaVine                -6.771429
Anthony Edwards            -5.000000
De'Aaron Fox               -4.450000
Miles Bridges              -4.366667
Jordan Poole               -3.750000
Devin Booker               -3.383333
RJ Barrett                 -3.200000
LaMelo Ball                -3.100000
Terry Rozier               -2.716667
Jerami Grant               -2.671429
Gary Trent Jr.             -2.633333
Brandon Ingram             -2.500000
Tyler Herro                -2.300000
Shai Gilgeous-Alexander    -2.066667
Trae Young                 -2.000000
Dillon Brooks              -0.600000
Karl-Anthony Towns          0.400000
Bradley Beal                0.785714
Kristaps Porzingis          0.940000
Ja Morant                   1.000000
Dejounte Murray             1.325000
Bam Adebayo                 1.375000
Domantas Sabonis            1.600000
DeMar DeRozan               1.757143
Luka Doncic                 2.0

In [211]:
commonNew = common[((common["season"].str.slice(start=0,stop=4)).astype(int) > 2015) & ((common["season"].str.slice(start=0,stop=4)).astype(int) < 2021)]

uniques = np.array(commonNew["team-year"].unique())
all_Pairs = [[]]*len(uniques)
for i in range(0,len(uniques)):
    all_Pairs[i] = list(commonNew[commonNew["team-year"] == uniques[i]]["PLAYER"])

unique_players = [ 'Kristaps Porzingis', 'Luka Doncic','Bradley Beal',
       'Jayson Tatum', 'Terry Rozier', 'LaMelo Ball', 'Donovan Mitchell', 'Anthony Davis','LeBron James', 
        'Kyrie Irving', 'Jaylen Brown', 'Malcolm Brogdon',
       'Miles Bridges', 'Darius Garland', 'Stephen Curry',
       'Kevin Durant', 'Russell Westbrook', 'Jerami Grant',
       'Paul George', 'Giannis Antetokounmpo', 'Khris Middleton', 'Klay Thompson',
       "James Harden", "Brandon Ingram", 'Nikola Jokic',"Julius Randle",
       'RJ Barrett', 'Shai Gilgeous-Alexander', 'Jordan Poole', 'Devin Booker',
       'Joel Embiid', 'Zach LaVine', 'Norman Powell',
       'DeMar DeRozan', 'Trae Young', 'Anthony Edwards',
       'Karl-Anthony Towns', 'Domantas Sabonis', 'Damian Lillard',  'Fred VanVleet', 'Dejounte Murray',
       'Dillon Brooks', 'Tyler Herro','Jimmy Butler',
       "De'Aaron Fox", "CJ McCollum", 'Gary Trent Jr.', 'Pascal Siakam',
        'Ja Morant', 'Bam Adebayo']

# Step 2: Create an empty matrix (DataFrame) with player names as both rows and columns
matrix = pd.DataFrame(0, index=unique_players, columns=unique_players)

# Step 3: Iterate through the list of arrays and update the counts in the matrix
for players in all_Pairs:
    for i in range(len(players)):
        for j in range(i + 1, len(players)):
            matrix.loc[players[i], players[j]] += 1
            matrix.loc[players[j], players[i]] += 1

# Optional: convert the DataFrame to int for better readability
#matrix = matrix.astype(int)
matrix.index.name = "Player"

matrix.to_csv("AllConnections.csv")

In [212]:
common22 = common[common["season"] == "2021-22"]

uniques = np.array(common22["team-year"].unique())
all_Pairs = [[]]*len(uniques)
for i in range(0,len(uniques)):
    all_Pairs[i] = list(common22[common22["team-year"] == uniques[i]]["PLAYER"])

unique_players = [ 'Kristaps Porzingis', 'Luka Doncic','Bradley Beal',
       'Jayson Tatum', 'Terry Rozier', 'LaMelo Ball', 'Donovan Mitchell', 'Anthony Davis','LeBron James', 
        'Kyrie Irving', 'Jaylen Brown', 'Malcolm Brogdon',
       'Miles Bridges', 'Darius Garland', 'Stephen Curry',
       'Kevin Durant', 'Russell Westbrook', 'Jerami Grant',
       'Paul George', 'Giannis Antetokounmpo', 'Khris Middleton', 'Klay Thompson',
       "James Harden", "Brandon Ingram", 'Nikola Jokic',"Julius Randle",
       'RJ Barrett', 'Shai Gilgeous-Alexander', 'Jordan Poole', 'Devin Booker',
       'Joel Embiid', 'Zach LaVine', 'Norman Powell',
       'DeMar DeRozan', 'Trae Young', 'Anthony Edwards',
       'Karl-Anthony Towns', 'Domantas Sabonis', 'Damian Lillard',  'Fred VanVleet', 'Dejounte Murray',
       'Dillon Brooks', 'Tyler Herro','Jimmy Butler',
       "De'Aaron Fox", "CJ McCollum", 'Gary Trent Jr.', 'Pascal Siakam',
        'Ja Morant', 'Bam Adebayo']

# Step 2: Create an empty matrix (DataFrame) with player names as both rows and columns
matrix = pd.DataFrame(0, index=unique_players, columns=unique_players)

# Step 3: Iterate through the list of arrays and update the counts in the matrix
for players in all_Pairs:
    for i in range(len(players)):
        for j in range(i + 1, len(players)):
            matrix.loc[players[i], players[j]] += 1
            matrix.loc[players[j], players[i]] += 1

# Optional: convert the DataFrame to int for better readability
#matrix = matrix.astype(int)
matrix.index.name = "Player"

matrix.to_csv("2022Connections.csv")

In [213]:
stats = pd.read_csv("2022stats.csv")
stats

,index,player_name,team_abbreviation,age,gp,pts,reb,ast,net_rating,usg_pct,ts_pct,ast_pct,season
0,5,Joel Embiid,PHI,28.0,68,30.6,11.7,4.2,7.9,0.375,0.616,0.236,2021-22
1,25,LeBron James,LAL,37.0,56,30.3,8.2,6.2,-2.5,0.317,0.619,0.297,2021-22
2,35,Giannis Antetokounmpo,MIL,27.0,67,29.9,11.6,5.8,8.0,0.340,0.633,0.304,2021-22
3,50,Kevin Durant,BKN,33.0,55,29.9,7.4,6.4,6.5,0.310,0.634,0.274,2021-22
4,55,Luka Doncic,DAL,23.0,65,28.4,9.1,8.7,3.5,0.368,0.571,0.458,2021-22
5,60,Trae Young,ATL,23.0,76,28.4,3.7,9.7,2.3,0.333,0.603,0.442,2021-22
6,74,DeMar DeRozan,CHI,32.0,76,27.9,5.2,4.9,1.1,0.308,0.590,0.228,2021-22
7,86,Kyrie Irving,BKN,30.0,29,27.4,4.4,5.8,5.2,0.283,0.592,0.235,2021-22
8,90,Ja Morant,MEM,22.0,57,27.4,5.7,6.7,4.1,0.330,0.575,0.330,2021-22
9,98,Nikola Jokic,DEN,27.0,74,27.1,13.8,7.9,8.4,0.309,0.661,0.388,2021-22
